In [1]:
import tensorflow as tf
import os
import random
import shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from shutil import copyfile
import urllib.request



In [2]:
nrows = 4
ncols = 4

pic_index = 0 # Index for iterating over images

In [3]:
train_dir="seg_train/seg_train"
#print(len(os.listdir(train_dir)))
train_datagen=ImageDataGenerator(rescale=1.0/255,
                                rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                fill_mode='nearest')
train_generator=train_datagen.flow_from_directory(train_dir,
                                                 batch_size=10,
                                                 class_mode='binary',
                                                 target_size=(224,224))


Found 14034 images belonging to 6 classes.


In [4]:
test_dir="seg_test/seg_test"
#print(len(os.listdir(test_dir)))
test_datagen=ImageDataGenerator(rescale=1.0/255.)
test_generator=test_datagen.flow_from_directory(test_dir,
                                                 batch_size=10,
                                                 class_mode='binary',
                                                 target_size=(224,224))

Found 3000 images belonging to 6 classes.


In [5]:
pred_dir="seg_pred/seg_pred"
#print(len(os.listdir(test_dir)))
pred_datagen=ImageDataGenerator(rescale=1.0/255.)
pred_generator=pred_datagen.flow_from_directory(pred_dir,
                                                 batch_size=10,
                                                 class_mode='binary',
                                                 target_size=(150,150))

Found 0 images belonging to 0 classes.


In [6]:
#@title Visualization Utilities[RUN ME]
#Matplotlib config
plt.rc('image', cmap='gray')
plt.rc('grid', linewidth=0)
plt.rc('xtick', top=False, bottom=False, labelsize='large')
plt.rc('ytick', left=False, right=False, labelsize='large')
plt.rc('axes', facecolor='F8F8F8', titlesize="large", edgecolor='white')
plt.rc('text', color='a8151a')
plt.rc('figure', facecolor='F0F0F0')# Matplotlib fonts
MATPLOTLIB_FONT_DIR = os.path.join(os.path.dirname(plt.__file__), "mpl-data/fonts/ttf")
# utility to display a row of digits with their predictions
def display_images(digits, predictions, labels, title):

  n = 10

  indexes = np.random.choice(len(predictions), size=n)
  n_digits = digits[indexes]
  n_predictions = predictions[indexes]
  n_predictions = n_predictions.reshape((n,))
  n_labels = labels[indexes]
 
  fig = plt.figure(figsize=(20, 4))
  plt.title(title)
  plt.yticks([])
  plt.xticks([])

  for i in range(10):
    ax = fig.add_subplot(1, 10, i+1)
    class_index = n_predictions[i]
    
    plt.xlabel(classes[class_index])
    plt.xticks([])
    plt.yticks([])
    plt.imshow(n_digits[i])

# utility to display training and validation curves
def plot_metrics(metric_name, title, ylim=5):
  plt.title(title)
  plt.ylim(0,ylim)
  plt.plot(history.history[metric_name],color='blue',label=metric_name)
  plt.plot(history.history['val_' + metric_name],color='green',label='val_' + metric_name)

In [7]:
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  output_ims = tf.keras.applications.resnet50.preprocess_input(input_images)
  return output_ims


In [8]:
def feature_extractor(inputs):

  feature_extractor = tf.keras.applications.resnet.ResNet50(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
  return feature_extractor

In [9]:
def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(6, activation="softmax", name="classification")(x)
    return x

In [10]:
def final_model(inputs):

    #resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)

    resnet_feature_extractor = feature_extractor(inputs)
    classification_output = classifier(resnet_feature_extractor)

    return classification_output

In [11]:
def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(224,224,3))
  
  classification_output = final_model(inputs) 
  model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
  model.compile(optimizer='SGD', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model

In [12]:
model = define_compile_model()


In [13]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
classification (Dense)       (None, 6)                 3078  

In [ ]:
model.fit(train_generator,validation_data=test_generator,epochs=10)

Epoch 1/10
1404/1404 [==============================] - 9920s 7s/step - loss: 0.4610 - accuracy: 0.8295 - val_loss: 0.3505 - val_accuracy: 0.8837
Epoch 2/10
1404/1404 [==============================] - 10171s 7s/step - loss: 0.3056 - accuracy: 0.8897 - val_loss: 0.2363 - val_accuracy: 0.9133
Epoch 3/10
1404/1404 [==============================] - 9471s 7s/step - loss: 0.2524 - accuracy: 0.9078 - val_loss: 0.2431 - val_accuracy: 0.9120
Epoch 4/10
1404/1404 [==============================] - 9011s 6s/step - loss: 0.2265 - accuracy: 0.9189 - val_loss: 0.3734 - val_accuracy: 0.8877
Epoch 5/10
 290/1404 [=====>........................] - ETA: 1:37:29 - loss: 0.2051 - accuracy: 0.9286